In [4]:
class Solver:
    def __init__(self):
        pass

In [5]:

from __future__ import print_function
from fenics import *
import mshr

%matplotlib inline

T = 5.0            # final time
num_steps = 500    # number of time steps
dt = T / num_steps # time step size
eps = 0.01         # diffusion coefficient
K = 10.0           # reaction rate
#nx = 100
#ny = 100

# Read mesh from file
#mesh = RectangleMesh(Point(0, 0), Point(2.2, 0.41), nx, ny)

grid = mshr.Rectangle(Point(0, 0), Point(2.5, 2.5))
mesh = mshr.generate_mesh(grid, 64) 

# Define function space for velocity
W = VectorFunctionSpace(mesh, 'P', 2)

# Define function space for system of concentrations
P1 = FiniteElement('P', triangle, 1)
element = MixedElement(P1, P1, P1, P1)
V = FunctionSpace(mesh, element)

# Define test functions
v_1, v_2, v_3, v_4 = TestFunctions(V)

# Define functions for velocity and concentrations
w = Function(W)
u = Function(V)
u_n = Function(V)

# Split system functions to access components
u_1, u_2, u_3, u_4 = split(u)
u_n1, u_n2, u_n3, u_n4 = split(u_n)

# Define source terms
f_1 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.1,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)
f_2 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)
f_3 = Constant(0)

f_4 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.5,2)<0.05*0.05 ? 0.1 : 0',
                 degree=1)

# Define expressions used in variational forms
k = Constant(dt)
K = Constant(K)
eps = Constant(eps)

# Define variational problem
#F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
#  + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx  \
#  + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
#  + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx  \
#  + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
#  + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
#  - f_1*v_1*dx - f_2*v_2*dx - f_3*v_3*dx


F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
    + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
    + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
    + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
    + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
    + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
    +((u_4 - u_n4) / k)*v_4*dx + dot(w, grad(u_4))*v_4*dx \
    + eps*dot(grad(u_4), grad(v_4))*dx - f_1*v_1*dx - f_2*v_2*dx \
    - f_3*v_3*dx - f_4*v_4*dx
                



# Create time series for reading velocity data
timeseries_w = TimeSeries('Downloads/wind_data21')
#timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series')
  
    
# Create VTK files for visualization output
vtkfile_u_1 = File('reaction_system/u_1.pvd')
vtkfile_u_2 = File('reaction_system/u_2.pvd')
vtkfile_u_3 = File('reaction_system/u_3.pvd')
vtkfile_u_4 = File('reaction_system/u_4.pvd')


# Create progress bar
progress = Progress('Time-stepping')
#set_log_level(PROGRESS)

# Time-stepping
t = 0
for n in range(num_steps):

    # Update current time
    t += dt

    # Read velocity from file
    timeseries_w.retrieve(w.vector(), t)

    # Solve variational problem for time step
    solve(F == 0, u)

    # Save solution to file (VTK)
    _u_1, _u_2, _u_3, _u_4 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    vtkfile_u_3 << (_u_3, t)
    vtkfile_u_4 << (_u_4, t)
    
    # Update previous solution
    u_n.assign(u)

    # Update progress bar
#progress.update(t / T)

# Hold plot
#interactive()